In [2]:
import sys
sys.path.append('..')

from Model_Code.iohmm_nn_model import IOHMM_NN
from misc_fns import *

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

import pandas as pd

In [3]:
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)

In [4]:
inputs = torch.randn(1000, 1)
state = 0

In [5]:
targets = []
for input in inputs:
    prob = torch.sigmoid(input)
    change = (torch.rand(1) < prob).item()
    if change:
        state = 1 - state
    
    output = input * 5 * (1 - 2*state)
    targets.append([output])

targets = torch.tensor(targets, dtype=torch.float32)

In [6]:
input_dim  = 1
output_dim = 1
num_states = 2

model = IOHMM_NN(input_dim, output_dim, num_states=num_states, add_bias = False)

In [7]:
optimizer = optim.Adam(model.parameters(), lr=0.1)
loss_fn   = nn.MSELoss()

In [8]:
optimize(model, inputs, targets, optimizer, loss_fn)

Epoch 100, Loss: 26.082406997680664
Epoch 200, Loss: 26.072418212890625
Epoch 300, Loss: 26.069042205810547
Epoch 400, Loss: 26.06932830810547
Epoch 500, Loss: 26.067367553710938
Epoch 600, Loss: 26.068355560302734
Epoch 700, Loss: 26.067089080810547
Epoch 800, Loss: 26.0670108795166
Epoch 900, Loss: 26.067035675048828
Epoch 1000, Loss: 26.066688537597656


In [9]:
torch.save(model, "../Model/iohmm_nn_small.pth")

In [10]:
model.W_z[:,0,:], model.W_z[:,1,:]

(tensor([[ 0.3053],
         [-0.0069]], grad_fn=<SliceBackward0>),
 tensor([[-0.4369],
         [-0.3805]], grad_fn=<SliceBackward0>))

In [11]:
model.W_x[:,0,:].item(), model.W_x[:,1,:].item()

(4.885567665100098, -4.956416606903076)

In [12]:
model.W_pi

Parameter containing:
tensor([[-1.7116],
        [ 0.9817]], requires_grad=True)

In [16]:
inputs[0].item(), F.softmax(model.W_pi*inputs[0].item(), dim=0)

(-1.1258398294448853,
 tensor([[0.9540],
         [0.0460]], grad_fn=<SoftmaxBackward0>))

In [14]:
((model.W_z[0,0,0] - model.W_z[1,0,0]).item(),
 (model.W_z[0,1,0] - model.W_z[1,1,0]).item())

(0.3122044503688812, -0.05640628933906555)